# Recommendations in Keras using triplet loss
Along the lines of BPR [1]. 

[1] Rendle, Steffen, et al. "BPR: Bayesian personalized ranking from implicit feedback." Proceedings of the Twenty-Fifth Conference on Uncertainty in Artificial Intelligence. AUAI Press, 2009.

This is implemented (more efficiently) in LightFM (https://github.com/lyst/lightfm). See the MovieLens example (https://github.com/lyst/lightfm/blob/master/examples/movielens/example.ipynb) for results comparable to this notebook.

## Set up the architecture
A simple dense layer for both users and items: this is exactly equivalent to latent factor matrix when multiplied by binary user and item indices. There are three inputs: users, positive items, and negative items. In the triplet objective we try to make the positive item rank higher than the negative item for that user.

Because we want just one single embedding for the items, we use shared weights for the positive and negative item inputs (a siamese architecture).

This is all very simple but could be made arbitrarily complex, with more layers, conv layers and so on. I expect we'll be seeing a lot of papers doing just that.


In [32]:
"""
Triplet loss network example for recommenders
"""

from __future__ import print_function

import numpy as np

from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.engine.topology import Layer

import data
import metrics


class BprLoss(Layer):
    
    def __init__(self, **kwargs):
        self.support_mask = True
        super(BprLoss, self).__init__(**kwargs)

    def build(self, input_shape):
        super(BprLoss, self).build(input_shape)
        
    def call(self, inputs):
        
        assert len(inputs) == 3
        pos_item = inputs[0]
        neg_item = inputs[1]
        user = inputs[2]
        
        loss = 1.0 - K.sigmoid(
            K.sum(user * pos_item, axis=-1, keepdims=True) -
            K.sum(user * neg_item, axis=-1, keepdims=True))

        return loss


def identity_loss(y_true, y_pred):

    return K.mean(y_pred * y_true)


def build_model(num_users, num_items, latent_dim):

    positive_item_input = Input((1, ), name='positive_item_input')
    negative_item_input = Input((1, ), name='negative_item_input')

    # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        num_items, latent_dim, name='item_embedding', input_length=1)

    user_input = Input((1, ), name='user_input')

    positive_item_embedding = Flatten()(item_embedding_layer(
        positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(
        negative_item_input))
    user_embedding = Flatten()(Embedding(
        num_users, latent_dim, name='user_embedding', input_length=1)(
            user_input))

    loss = BprLoss(name='bpr_loss')([positive_item_embedding, negative_item_embedding, user_embedding])

    model = Model(
        inputs=[positive_item_input, negative_item_input, user_input],
        outputs=loss)
    model.compile(loss=identity_loss, optimizer=Adam())

    return model

## Load and transform data
We're going to load the Movielens 100k dataset and create triplets of (user, known positive item, randomly sampled negative item).

The success metric is AUC: in this case, the probability that a randomly chosen known positive item from the test set is ranked higher for a given user than a ranomly chosen negative item.

In [33]:
latent_dim = 100
num_epochs = 10

# Read data
train, test = data.get_movielens_data()
num_users, num_items = train.shape

# Prepare the test triplets
test_uid, test_pid, test_nid = data.get_triplets(test)

model = build_model(num_users, num_items, latent_dim)

# Print the model structure
print(model.summary())

# Sanity check, should be around 0.5
print('AUC before training %s' % metrics.full_auc(model, test))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
positive_item_input (InputLayer) (None, 1)             0                                            
____________________________________________________________________________________________________
negative_item_input (InputLayer) (None, 1)             0                                            
____________________________________________________________________________________________________
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
item_embedding (Embedding)       (None, 1, 100)        168300      positive_item_input[0][0]        
                                                                   negative_item_input[0][0

## Run the model
Run for a couple of epochs, checking the AUC after every epoch.

In [34]:
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = data.get_triplets(train)

    X = {
        'user_input': uid,
        'positive_item_input': pid,
        'negative_item_input': nid
    }

    model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              epochs=1,
              verbose=0,
              shuffle=True)

    print('AUC %s' % metrics.full_auc(model, test))

Epoch 0
AUC 0.832828481741
Epoch 1
AUC 0.854822516458
Epoch 2
AUC 0.863386414061
Epoch 3
AUC 0.871770897517
Epoch 4
AUC 0.879935496328
Epoch 5
AUC 0.886749376039
Epoch 6
AUC 0.893268786229
Epoch 7
AUC 0.897671810193
Epoch 8
AUC 0.901398281636
Epoch 9
AUC 0.904418201168


The AUC is in the low-90s. At some point we start overfitting, so it would be a good idea to stop early or add some regularization.